In [ ]:
import os
import pandas as pd
import zipfile
import io

folder = os.path.join(r"C:/Users/Ignac/Trabajo_Centra/Catedra-LDES/CII-Centra-EDF/Operacion", r"escenarios/CaseBase1/RL/")
years = [2029, 2030, 2031, 2033, 2040, 2050]
# years = [2050]
# Days = [0]
Days = vector = list(range(0, 365))

filenames_outputs1 = ["storage_dispatch.csv","TES_dispatch.txt","DispatchGen.csv","dispatch.csv","UnservedLoad.csv","dispatch_annual_summary.csv","marginal_cost.csv"]
filenames_inputs = ["variable_capacity_factors.csv","loads.csv"]

counter = 0

nombres_columnas = [
    "Day",
    "Capacidad instalada Generación",
    "Capacidad instalada LDES",
    "Capacidad instalada BESS",
    "Participación renovable",
    "Participación renovable variable",
    "Generación bruta",
    "Energía movida por LDES",
    "Energía movida por BESS",
    "Generación renovable energía",
    "Generación renovable variable energía",
    "Participación renovable energía",
    "Participación renovable variable energía",
    "Renovable_potencial",
    "Recortes",
    "Recortes %",
    "Energía no servida",
    "Energía total demandada (GWh)",
    "Costos marginales"
]

LDES_list = ["Bomb", "TES", "CAES","CSP-TES"]
BESS_list = ["ESS"]
Renovables_list = ["Hidroelectrica","Solar_FV","Biogas","Biomasa","Geotermica","Eolica"]
Solar_y_Eolica_list =["Solar_FV","Eolica"]

nombre_columnas_anualls = ["Año",
    "Capacidad instalada Generación",
    "Capacidad instalada LDES",
    "Capacidad instalada BESS",
    "Participación renovable",
    "Participación renovable variable",
    "Generación bruta",
    "Energía movida por LDES",
    "Energía movida por BESS",
    "Generación renovable energía",
    "Generación renovable variable energía",
    "Participación renovable energía",
    "Participación renovable variable energía",
    "Recortes",
    "Recortes %",
    "Energía no servida",
    "Energía total demandada (GWh)",
    "Costos marginales"]

df_energy_anualls = pd.DataFrame(columns=nombre_columnas_anualls)

for year in years:
    dataframe_year = pd.DataFrame(columns=nombres_columnas)
    year_str = str(year)
    rute = "inputs_" + year_str + ".zip"
    zip_path = os.path.join(folder, rute)
    try:
        with zipfile.ZipFile(zip_path, 'r') as archive:
            for day in Days:
                dataframes_outputs = {}
                dataframes_inputs = {}
                day_str = str(day)
                rute_inputs = "inputs_" + year_str + "/" + day_str + "/inputs_dispatch"
                rute_outputs = "inputs_" + year_str + "/" + day_str + "/outputs_dispatch"            
                
                # para archivos en carpeta outputs
                for nombre_archivo in filenames_outputs1:
                    ruta_completa = rute_outputs + "/" + nombre_archivo
                    try:
                        with archive.open(ruta_completa, 'r') as archivo_interno:
                            # Leer el contenido como bytes y luego usar io.BytesIO para que pandas lo lea
                            contenido_bytes = archivo_interno.read()
                            df = pd.read_csv(io.BytesIO(contenido_bytes))
                            dataframes_outputs[nombre_archivo] = df
                            # print(f"DataFrame creado para '{nombre_archivo}'")
                    except KeyError:
                        print(f"Error: El archivo '{ruta_completa}' no fue encontrado dentro del ZIP.")
                    except Exception as e:
                        print(f"Error al leer '{nombre_archivo}': {e}")

                # para archivos en carpeta inputs
                for nombre_archivo in filenames_inputs:
                    ruta_completa = rute_inputs + "/" + nombre_archivo
                    try:
                        with archive.open(ruta_completa, 'r') as archivo_interno:
                            # Leer el contenido como bytes y luego usar io.BytesIO para que pandas lo lea
                            contenido_bytes = archivo_interno.read()
                            df = pd.read_csv(io.BytesIO(contenido_bytes))
                            dataframes_inputs[nombre_archivo] = df
                            # print(f"DataFrame creado para '{nombre_archivo}'")
                    except KeyError:
                        print(f"Error: El archivo '{ruta_completa}' no fue encontrado dentro del ZIP.")
                    except Exception as e:
                        print(f"Error al leer '{nombre_archivo}': {e}")
                    # Ahora el diccionario 'dataframes' contiene los DataFrames, donde la clave es el nombre del archivo


                # calculo inicial necesario, día del año actual.
                df_load = dataframes_inputs["loads.csv"]
                df_load["TIMEPOINT"] = df_load["TIMEPOINT"].astype(str)
                dia_actual_str = df_load["TIMEPOINT"][1][6:8]

                # Calculos energéticos
                # 1. Demanda del sistema
                df_load = dataframes_inputs["loads.csv"]
                # df_load["TIMEPOINT"] = df_load["TIMEPOINT"].astype(str)
                df_load = df_load[df_load['TIMEPOINT'].astype(str).apply(lambda x: x[6:8] == dia_actual_str if len(x) >= 8 else False)]
                load = df_load["zone_demand_mw"].sum()/1000


                # 2. Energía movida por BESS
                # Crear un patrón regex combinando las subcadenas a excluir
                patron_exclusion = '|'.join(LDES_list)
                df_storage = dataframes_outputs["storage_dispatch.csv"]
                # Filtrar el DataFrame para que no contenga ninguna de las subcadenas
                df_storage = df_storage[~df_storage['generation_project'].str.contains(patron_exclusion, case=False, na=False)]
                df_storage = df_storage[df_storage['timepoint'].apply(lambda x: x[8:10] == dia_actual_str if len(x) >= 10 else False)]
                BESS_energy = df_storage["DischargeMW"].sum()/1000


                # 3. Energía movida por LDES
                patron_exclusion = '|'.join(LDES_list)
                df_storage = dataframes_outputs["storage_dispatch.csv"]
                df_storage = df_storage[df_storage['generation_project'].str.contains(patron_exclusion, case=False, na=False)]
                df_storage = df_storage[df_storage['timepoint'].apply(lambda x: x[8:10] == dia_actual_str if len(x) >= 10 else False)]
                LDES_energy1 = df_storage["DischargeMW"].sum()/1000
                
                # TES está por separado
                df_tes = dataframes_outputs["TES_dispatch.txt"]
                tes_energy = 0
                if not df_tes.empty:
                    df_tes = df_tes[df_tes['timepoint'].apply(lambda x: x[6:8] == dia_actual_str if len(x) >= 8 else False)]
                    tes_energy = df_tes["DischargeTES_MWt"].sum()/1000

                LDES_energy = LDES_energy1 + tes_energy


                # 4. Capacidad instalada
                df_capacity = dataframes_outputs["dispatch_annual_summary.csv"]
                gen_capacity = df_capacity["GenCapacity_MW"].sum()/1000
                # print(gen_capacity)

                # 5. Capacidad instalada LDES
                patron_exclusion = '|'.join(LDES_list)
                df_capacity1 = df_capacity[df_capacity['gen_energy_source'].str.contains(patron_exclusion, case=False, na=False)]
                LDES_capacity = df_capacity1["GenCapacity_MW"].sum()/1000
                # print(LDES_capacity)

                # 6. Capacidad instalada BESS
                patron_exclusion_Bess = '|'.join(BESS_list)
                df_capacity2 = df_capacity[df_capacity['gen_energy_source'].str.contains(patron_exclusion_Bess, case=False, na=False)]
                BESS_capacity = df_capacity2["GenCapacity_MW"].sum()/1000
                # print(BESS_capacity)


                # 7. Participación Renovable (Capaciad)
                patron_renovable = '|'.join(Renovables_list)
                df_capacity3 = df_capacity[df_capacity['gen_energy_source'].str.contains(patron_renovable, case=False, na=False)]
                renovable_capacity = df_capacity3["GenCapacity_MW"].sum()/1000
                renovable_porcentaje = 100*renovable_capacity/gen_capacity
                # print(renovable_capacity)

                # 8. Participación Renovable Variable (Capacidad)
                patron_sol_y_viento = '|'.join(Solar_y_Eolica_list)
                df_capacity4 = df_capacity[df_capacity['gen_energy_source'].str.contains(patron_sol_y_viento, case=False, na=False)]
                sol_y_viento_capacity = df_capacity4["GenCapacity_MW"].sum()/1000
                sol_y_viento_capacity_porcentaje = 100*sol_y_viento_capacity/gen_capacity
                # print(sol_y_viento_capacity)

                # 9. Generación Bruta (Energía)
                df_dispatch = dataframes_outputs["dispatch.csv"]
                df_dispatch["timestamp"] = df_dispatch["timestamp"].astype(str)
                df_dispatch = df_dispatch[df_dispatch['timestamp'].apply(lambda x: x[8:10] == dia_actual_str if len(x) >= 10 else False)]
                generation = df_dispatch["DispatchGen_MW"].sum()/1000
                # print(generation)

                # 10 Energía generada por Renovables (Energía)
                df_dispatch = dataframes_outputs["dispatch.csv"]
                df_dispatch["timestamp"] = df_dispatch["timestamp"].astype(str)
                df_dispatch = df_dispatch[df_dispatch['timestamp'].apply(lambda x: x[8:10] == dia_actual_str if len(x) >= 10 else False)]
                df_dispatch = df_dispatch[df_dispatch['gen_energy_source'].str.contains(patron_renovable, case=False, na=False)]
                generation_renovable = df_dispatch["DispatchGen_MW"].sum()/1000
                # print(generation_renovable)

                # 11 Energía generada por Renovables Variables (Energía)
                df_dispatch = dataframes_outputs["dispatch.csv"]
                df_dispatch["timestamp"] = df_dispatch["timestamp"].astype(str)
                df_dispatch = df_dispatch[df_dispatch['timestamp'].apply(lambda x: x[8:10] == dia_actual_str if len(x) >= 10 else False)]
                df_dispatch = df_dispatch[df_dispatch['gen_energy_source'].str.contains(patron_sol_y_viento, case=False, na=False)]
                generation_sol_y_viento = df_dispatch["DispatchGen_MW"].sum()/1000
                # print(generation_sol_y_viento)

                # 12. Energía no servida
                df_unserved_load = dataframes_outputs["UnservedLoad.csv"]
                df_unserved_load["SetProduct_OrderedSet_2"] = df_unserved_load["SetProduct_OrderedSet_2"].astype(str)
                df_unserved_load = df_unserved_load[df_unserved_load['SetProduct_OrderedSet_2'].apply(lambda x: x[6:8] == dia_actual_str if len(x) >= 8 else False)]
                unserved_energy = df_unserved_load["UnservedLoad"].sum()/1000
                # print(unserved_energy)
                

                # 13. Recortes solares y eolicos (Energía)
                df_dispatch = dataframes_outputs["dispatch.csv"]
                df_dispatch["timestamp"] = df_dispatch["timestamp"].astype(str)
                df_dispatch = df_dispatch[df_dispatch['timestamp'].apply(lambda x: x[8:10] == dia_actual_str if len(x) >= 10 else False)]
                df_dispatch = df_dispatch[df_dispatch['gen_energy_source'].str.contains(patron_sol_y_viento, case=False, na=False)]
                df_dispatch['generation_project'] = df_dispatch['generation_project'].replace('Las AraÃ±as 1 13.2 II', 'Las Arañas 1 13.2 II')

                pv_eo_gens = df_dispatch["generation_project"].unique()
                # print(pv_eo_gens)
                df_variable_capacity_factors = dataframes_inputs["variable_capacity_factors.csv"]
                # print(df_variable_capacity_factors)
                df_variable_capacity_factors = df_variable_capacity_factors[df_variable_capacity_factors["GENERATION_PROJECT"].isin(pv_eo_gens)]
                # df_variable_capacity_factors["timepoint"] = df_variable_capacity_factors["timepoint"].astype(str)
                df_variable_capacity_factors = df_variable_capacity_factors[df_variable_capacity_factors["timepoint"].astype(str).apply(lambda x: x[6:8] == dia_actual_str if len(x) >= 8 else False)]
                df_variable_capacity_factors['GENERATION_PROJECT'] = df_variable_capacity_factors['GENERATION_PROJECT'].replace('Las AraÃ±as 1 13.2 II', 'Las Arañas 1 13.2')

                df_dispatch["gen_max_capacity_factor"] = pd.Series([0] * len(df_dispatch))
                # Itera sobre cada generador único
                cun = 0
                for generador in pv_eo_gens:
                    # Filtra las filas donde 'generation_project' coincide con el generador en df_dispatch
                    filas_dispatch = df_dispatch[df_dispatch["generation_project"] == generador]
                    
                    # Filtra las filas donde 'GENERATION_PROJECT' coincide con el generador en df_variable_capacity_factors
                    filas_generador_vcf = df_variable_capacity_factors[df_variable_capacity_factors["GENERATION_PROJECT"] == generador]
                    
                    # Verifica que ambas selecciones tengan la misma cantidad de filas (24 en tu caso)
                    if len(filas_dispatch) == 24 and len(filas_generador_vcf) == 24:
                        # Extrae los capacity_factors en el orden correcto.  Esto es una Serie de Pandas.
                        capacity_factors = filas_generador_vcf["gen_max_capacity_factor"].values  # .values entrega un array de NumPy
                        
                        # Actualiza df_dispatch usando .loc y los índices de las filas_dispatch
                        indices_dispatch = filas_dispatch.index
                        df_dispatch.loc[indices_dispatch, "gen_max_capacity_factor"] = capacity_factors
                        
                        # print("Asignación exitosa") # Añadí este print
                    else:
                        print(f"Error: El número de filas para el generador '{generador}' no es 24 en ambos DataFrames.  Omitiendo la asignación.")
                # print("hola")
                df_dispatch['Potential_gen'] = df_dispatch['GenCapacity_MW'].multiply(df_dispatch['gen_max_capacity_factor'])
                df_dispatch["Recortes"] = df_dispatch['Potential_gen'].subtract(df_dispatch["DispatchGen_MW"])

                Gen_potential = df_dispatch['Potential_gen'].sum()/1000
                Recortes_tot = df_dispatch["Recortes"].sum()/1000
                # print(Recortes_tot)


                # 15. Marginal Cost
                df_marg_cost = dataframes_outputs["marginal_cost.csv"]
                df_marg_cost = df_marg_cost[df_marg_cost["period"].astype(str).apply(lambda x: x[8:10] == dia_actual_str if len(x) >= 8 else False)]
                marg_cost = df_marg_cost["timepoint"].mean()

                # Crear un diccionario con los valores de la nueva fila
                nueva_fila = { 
                            "Day":day,
                            "Capacidad instalada Generación": gen_capacity,
                            "Capacidad instalada LDES": LDES_capacity, 
                           "Capacidad instalada BESS":BESS_capacity, 
                           "Participación renovable":renovable_porcentaje,
                           "Participación renovable variable":sol_y_viento_capacity_porcentaje,
                           "Generación bruta":generation,
                           "Energía movida por LDES":LDES_energy,
                           "Energía movida por BESS":BESS_energy,
                           "Generación renovable energía":generation_renovable,
                           "Generación renovable variable energía":generation_sol_y_viento,
                           "Participación renovable energía":0,
                           "Participación renovable variable energía":0,
                           "Renovable_potencial":Gen_potential,
                           "Recortes":Recortes_tot,
                           "Recortes %":0,
                           "Energía no servida":unserved_energy,
                           "Energía total demandada (GWh)":load,
                           "Costos marginales":marg_cost}

                # Convertir la nueva fila en un DataFrame
                nueva_fila_df = pd.DataFrame([nueva_fila])

                # Concatenar la nueva fila al DataFrame existente
                dataframe_year = dataframe_year.dropna(how='all')
                dataframe_year = pd.concat([dataframe_year, nueva_fila_df], ignore_index=True)
            display(dataframe_year)

    except FileNotFoundError:
        print(f"Error: El archivo ZIP '{zip_path}' no fue encontrado.")
    except Exception as e:
        print(f"Ocurrió un error al abrir el archivo ZIP: {e}")

    gen_capacity_anuall = dataframe_year["Capacidad instalada Generación"][0]
    LDES_capacity_anuall = dataframe_year["Capacidad instalada LDES"][0]
    BESS_capacity_anuall = dataframe_year["Capacidad instalada BESS"][0]
    renovable_porcentaje_anuall = dataframe_year["Participación renovable"][0]
    sol_y_viento_capacity_porcentaje_anuall = dataframe_year["Participación renovable variable"][0]
    generation_anuall = dataframe_year["Generación bruta"].sum()
    LDES_energy_anuall = dataframe_year["Energía movida por LDES"].sum()
    BESS_energy_anuall = dataframe_year["Energía movida por BESS"].sum()
    generation_renovable_anuall = dataframe_year["Generación renovable energía"].sum()
    generation_sol_y_viento_anuall = dataframe_year["Generación renovable variable energía"].sum()
    participacion_renovable_percent = 100*dataframe_year["Generación renovable energía"].sum()/generation_anuall
    participacion_sol_y_viento_percent = 100*dataframe_year["Generación renovable variable energía"].sum()/generation_anuall
    Recortes_tot_anuall = dataframe_year["Recortes"].sum()
    recortes_percent = 100*Recortes_tot_anuall/dataframe_year["Renovable_potencial"].sum()
    unserved_energy_anuall = dataframe_year["Energía no servida"].sum()
    load_anuall = dataframe_year["Energía total demandada (GWh)"].sum()
    marg_cost_anuall = dataframe_year["Costos marginales"].mean()
    
    nueva_fila_anualls = { "Año": year,
                            "Capacidad instalada Generación": gen_capacity_anuall,
                            "Capacidad instalada LDES": LDES_capacity_anuall, 
                            "Capacidad instalada BESS":BESS_capacity_anuall, 
                            "Participación renovable":renovable_porcentaje_anuall,
                            "Participación renovable variable":sol_y_viento_capacity_porcentaje_anuall,
                            "Generación bruta":generation_anuall,
                            "Energía movida por LDES":LDES_energy_anuall,
                            "Energía movida por BESS":BESS_energy_anuall,
                            "Generación renovable energía":generation_renovable_anuall,
                            "Generación renovable variable energía":generation_sol_y_viento_anuall,
                            "Participación renovable energía":participacion_renovable_percent,
                            "Participación renovable variable energía":participacion_sol_y_viento_percent,
                            "Recortes":Recortes_tot_anuall,
                            "Recortes %":recortes_percent,
                            "Energía no servida":unserved_energy_anuall,
                            "Energía total demandada (GWh)":load_anuall,
                            "Costos marginales":marg_cost_anuall}
    
    # Convertir la nueva fila en un DataFrame
    nueva_fila_df_anualls = pd.DataFrame([nueva_fila_anualls])
    # Concatenar la nueva fila al DataFrame existente
    df_energy_anualls = df_energy_anualls.dropna(how='all')
    df_energy_anualls = pd.concat([df_energy_anualls, nueva_fila_df_anualls], ignore_index=True)
    
display(df_energy_anualls)


C:\Users\Ignac\AppData\Local\Temp\ipykernel_4636\3593571445.py:292: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe_year = pd.concat([dataframe_year, nueva_fila_df], ignore_index=True)


,Day,Capacidad instalada Generación,Capacidad instalada LDES,Capacidad instalada BESS,Participación renovable,Participación renovable variable,Generación bruta,Energía movida por LDES,Energía movida por BESS,Generación renovable energía,Generación renovable variable energía,Participación renovable energía,Participación renovable variable energía,Renovable_potencial,Recortes,Recortes %,Energía no servida,Energía total demandada (GWh),Costos marginales
0,0,44.821459,0.275808,2.5006,66.34416,47.25027,256.718508,14.938394,5.433126,206.349150,135.285173,0,0,159.913223,24.628049,0,0.0,224.152034,18.683785
1,1,44.821459,0.275808,2.5006,66.34416,47.25027,256.861146,15.861910,6.006356,206.773586,136.679885,0,0,159.913223,23.233337,0,0.0,224.152034,17.972625
2,2,44.821459,0.275808,2.5006,66.34416,47.25027,256.588777,15.861910,5.973848,206.589047,135.932443,0,0,159.913223,23.980779,0,0.0,224.152034,17.972625
3,3,44.821459,0.275808,2.5006,66.34416,47.25027,256.251461,15.861909,6.028851,205.943465,133.095813,0,0,159.913223,26.817410,0,0.0,224.152034,17.972625
4,4,44.821459,0.275808,2.5006,66.34416,47.25027,256.881596,15.861910,6.034785,206.620376,134.674725,0,0,159.913223,25.238498,0,0.0,224.152034,17.972625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,360,44.821459,0.275808,2.5006,66.34416,47.25027,272.627615,14.463975,12.642078,210.624214,139.716088,0,0,141.730363,2.014275,0,0.0,226.800908,24.955895
361,361,44.821459,0.275808,2.5006,66.34416,47.25027,269.382330,15.090801,12.264872,208.488518,137.643384,0,0,141.730363,4.086979,0,0.0,226.800908,24.619174
362,362,44.821459,0.275808,2.5006,66.34416,47.25027,269.247557,15.140845,11.990084,208.583790,138.258242,0,0,141.730363,3.472121,0,0.0,226.800908,24.617080
363,363,44.821459,0.275808,2.5006,66.34416,47.25027,268.989194,15.383174,11.773153,208.314958,137.436350,0,0,141.730363,4.294013,0,0.0,226.800908,24.615834


C:\Users\Ignac\AppData\Local\Temp\ipykernel_4636\3593571445.py:341: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_energy_anualls = pd.concat([df_energy_anualls, nueva_fila_df_anualls], ignore_index=True)
C:\Users\Ignac\AppData\Local\Temp\ipykernel_4636\3593571445.py:292: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe_year = pd.concat([dataframe_year, nueva_fila_df], ignore_index=True)


,Day,Capacidad instalada Generación,Capacidad instalada LDES,Capacidad instalada BESS,Participación renovable,Participación renovable variable,Generación bruta,Energía movida por LDES,Energía movida por BESS,Generación renovable energía,Generación renovable variable energía,Participación renovable energía,Participación renovable variable energía,Renovable_potencial,Recortes,Recortes %,Energía no servida,Energía total demandada (GWh),Costos marginales
0,0,44.685659,0.275808,2.5006,66.545781,47.393864,257.294517,15.861908,5.907550,207.957207,134.887904,0,0,159.913223,25.025319,0,0.0,224.993200,18.777375
1,1,44.685659,0.275808,2.5006,66.545781,47.393864,258.123898,15.990423,5.974271,208.902355,136.014024,0,0,159.913223,23.899199,0,0.0,224.993200,18.777221
2,2,44.685659,0.275808,2.5006,66.545781,47.393864,258.322922,16.019751,6.008028,209.420056,134.962064,0,0,159.913223,24.951159,0,0.0,224.993200,18.777375
3,3,44.685659,0.275808,2.5006,66.545781,47.393864,256.923755,15.976192,6.020047,207.621381,131.825738,0,0,159.913223,28.087485,0,0.0,224.993200,18.777375
4,4,44.685659,0.275808,2.5006,66.545781,47.393864,257.252008,16.008849,5.998579,208.037382,132.514539,0,0,159.913223,27.398684,0,0.0,224.993200,18.777221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,360,44.685659,0.275808,2.5006,66.545781,47.393864,272.551173,15.859174,10.128454,196.586657,123.109048,0,0,124.777496,1.668447,0,0.0,227.651351,33.649754
361,361,44.685659,0.275808,2.5006,66.545781,47.393864,272.149652,16.699071,10.792121,196.586657,123.109048,0,0,124.777496,1.668447,0,0.0,227.651351,33.377474
362,362,44.685659,0.275808,2.5006,66.545781,47.393864,272.175388,16.334787,10.780939,196.586657,123.109048,0,0,124.777496,1.668447,0,0.0,227.651351,33.377474
363,363,44.685659,0.275808,2.5006,66.545781,47.393864,272.120611,16.215010,10.797826,196.586657,123.109048,0,0,124.777496,1.668447,0,0.0,227.651351,33.377474


C:\Users\Ignac\AppData\Local\Temp\ipykernel_4636\3593571445.py:292: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe_year = pd.concat([dataframe_year, nueva_fila_df], ignore_index=True)


...

C:\Users\Ignac\AppData\Local\Temp\ipykernel_4636\3593571445.py:292: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe_year = pd.concat([dataframe_year, nueva_fila_df], ignore_index=True)


,Day,Capacidad instalada Generación,Capacidad instalada LDES,Capacidad instalada BESS,Participación renovable,Participación renovable variable,Generación bruta,Energía movida por LDES,Energía movida por BESS,Generación renovable energía,Generación renovable variable energía,Participación renovable energía,Participación renovable variable energía,Renovable_potencial,Recortes,Recortes %,Energía no servida,Energía total demandada (GWh),Costos marginales
0,0,48.316181,1.93342,2.520943,72.266364,54.55354,322.896327,106.691322,5.838110,265.362879,191.102909,0,0,203.267291,12.164382,0,0.0,268.288712,11.683062
1,1,48.316181,1.93342,2.520943,72.266364,54.55354,322.182023,113.513040,6.234353,265.101510,192.087663,0,0,203.267291,11.179628,0,0.0,268.288712,11.257982
2,2,48.316181,1.93342,2.520943,72.266364,54.55354,338.057812,115.196417,12.538108,274.604239,199.493441,0,0,203.267291,3.773850,0,0.0,268.288712,11.257982
3,3,48.316181,1.93342,2.520943,72.266364,54.55354,322.419033,113.582255,6.246647,265.344139,192.918627,0,0,203.267291,10.348664,0,0.0,268.288712,11.250304
4,4,48.316181,1.93342,2.520943,72.266364,54.55354,337.707354,114.981213,11.890800,274.683209,199.535776,0,0,203.267291,3.731515,0,0.0,268.288712,11.257982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,360,48.316181,1.93342,2.520943,72.266364,54.55354,328.910712,90.124816,11.213745,258.935579,184.386992,0,0,186.359313,1.972321,0,0.0,271.249773,18.078206
361,361,48.316181,1.93342,2.520943,72.266364,54.55354,329.042333,89.575955,10.349369,259.078065,184.384699,0,0,186.359313,1.974614,0,0.0,271.249773,18.976237
362,362,48.316181,1.93342,2.520943,72.266364,54.55354,328.793778,100.248866,10.924959,258.872993,184.384699,0,0,186.359313,1.974614,0,0.0,271.249773,18.003499
363,363,48.316181,1.93342,2.520943,72.266364,54.55354,328.790287,103.313952,10.939714,258.872993,184.384699,0,0,186.359313,1.974614,0,0.0,271.249773,17.962236


C:\Users\Ignac\AppData\Local\Temp\ipykernel_4636\3593571445.py:292: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe_year = pd.concat([dataframe_year, nueva_fila_df], ignore_index=True)


,Day,Capacidad instalada Generación,Capacidad instalada LDES,Capacidad instalada BESS,Participación renovable,Participación renovable variable,Generación bruta,Energía movida por LDES,Energía movida por BESS,Generación renovable energía,Generación renovable variable energía,Participación renovable energía,Participación renovable variable energía,Renovable_potencial,Recortes,Recortes %,Energía no servida,Energía total demandada (GWh),Costos marginales
0,0,63.034144,4.406606,3.469428,73.126217,59.475933,410.689513,248.006639,7.785385,334.663246,269.906646,0,0,298.366942,28.460296,0,0.0,320.399721,7.391916
1,1,63.034144,4.406606,3.469428,73.126217,59.475933,411.169526,259.365220,9.781588,335.877428,270.711073,0,0,298.366942,27.655869,0,0.0,320.399721,5.838055
2,2,63.034144,4.406606,3.469428,73.126217,59.475933,410.667144,259.723529,9.947675,335.068823,269.033101,0,0,298.366942,29.333841,0,0.0,320.399721,5.838055
3,3,63.034144,4.406606,3.469428,73.126217,59.475933,410.714044,259.414982,9.956763,335.316101,270.352374,0,0,298.366942,28.014568,0,0.0,320.399721,5.838055
4,4,63.034144,4.406606,3.469428,73.126217,59.475933,410.504940,259.335605,10.045876,335.051570,270.342401,0,0,298.366942,28.024541,0,0.0,320.399721,5.838055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,360,63.034144,4.406606,3.469428,73.126217,59.475933,411.844135,203.747373,16.553809,279.951452,204.190526,0,0,206.844468,2.653942,0,0.0,322.430494,15.183034
361,361,63.034144,4.406606,3.469428,73.126217,59.475933,407.112851,221.076440,18.741201,279.571482,204.190526,0,0,206.844468,2.653942,0,0.0,322.430494,13.111013
362,362,63.034144,4.406606,3.469428,73.126217,59.475933,406.044216,227.124906,17.797910,279.571482,204.190526,0,0,206.844468,2.653942,0,0.0,322.430494,13.103186
363,363,63.034144,4.406606,3.469428,73.126217,59.475933,403.816538,235.035396,18.368855,279.571482,204.190526,0,0,206.844468,2.653942,0,0.0,322.430494,12.687939


,Año,Capacidad instalada Generación,Capacidad instalada LDES,Capacidad instalada BESS,Participación renovable,Participación renovable variable,Generación bruta,Energía movida por LDES,Energía movida por BESS,Generación renovable energía,Generación renovable variable energía,Participación renovable energía,Participación renovable variable energía,Recortes,Recortes %,Energía no servida,Energía total demandada (GWh),Costos marginales
0,2029,44.821459,0.275808,2.500600,66.344160,47.250270,96399.331505,5687.246342,3329.654689,72503.119602,48680.948810,75.211227,50.499260,4613.847441,8.657219,0.0,82256.550743,24.750640
1,2030,44.685659,0.275808,2.500600,66.545781,47.393864,96403.257503,5699.973791,3203.789691,72509.264519,47786.244405,75.214538,49.569118,4899.642044,9.299724,0.0,82565.383381,25.949522
2,2031,44.541339,0.275808,2.500600,66.761398,47.547427,98243.313531,5538.036471,3362.674762,73490.979416,48554.908441,74.805070,49.423118,4782.132211,8.965875,0.0,83886.854135,23.866685
3,2033,44.541339,0.275808,2.500600,66.761398,47.547427,100806.183257,5799.846013,3139.904851,74633.713578,49755.500848,74.036841,49.357588,4004.639568,7.449087,0.0,86857.677617,19.246245
4,2040,48.316181,1.933420,2.520943,72.266364,54.553540,119309.115502,37567.373172,3091.026345,93010.304668,67502.646034,77.957417,56.577945,2229.397002,3.197091,0.0,98421.902820,16.906257
5,2050,63.034144,4.406606,3.469428,73.126217,59.475933,151157.733220,88270.859008,4962.285316,116651.344777,91920.746337,77.171933,60.811144,5550.521934,5.694521,0.0,117353.162553,8.667074


In [29]:
# Suponiendo que ya tienes el DataFrame df_energy_anualls cargado

# 1. Utilizamos la función melt para "desapilar" las columnas
df_melted = pd.melt(df_energy_anualls, id_vars=['Año'],
                    value_vars=[
                        "Capacidad instalada Generación",
                        "Capacidad instalada LDES",
                        "Capacidad instalada BESS",
                        "Participación renovable",
                        "Participación renovable variable",
                        "Generación bruta",
                        "Energía movida por LDES",
                        "Energía movida por BESS",
                        "Generación renovable energía",
                        "Generación renovable variable energía",
                        "Participación renovable energía",
                        "Participación renovable variable energía",
                        "Recortes",
                        "Recortes %",
                        "Energía no servida",
                        "Energía total demandada (GWh)",
                        "Costos marginales",
                    ],
                    var_name='item',
                    value_name='value')

# 2. Pivotamos la tabla resultante
df_transformed = df_melted.pivot_table(index='item', columns='Año', values='value', aggfunc='first')

# 3. Reiniciamos el índice para que 'item' sea una columna
df_transformed = df_transformed.reset_index()

# Definimos el orden deseado de las filas
orden_deseado = [
    "Capacidad instalada Generación",
    "Capacidad instalada LDES",
    "Capacidad instalada BESS",
    "Participación renovable",
    "Participación renovable variable",
    "Generación bruta",
    "Energía movida por LDES",
    "Energía movida por BESS",
    "Generación renovable energía",
    "Generación renovable variable energía",
    "Participación renovable energía",
    "Participación renovable variable energía",
    "Recortes",
    "Recortes %",
    "Energía no servida",
    "Energía total demandada (GWh)",
    "Costos marginales",
]

# 4. Reindexamos el DataFrame para aplicar el orden deseado
df_transformed = df_transformed.set_index('item').loc[orden_deseado].reset_index()

# Imprimimos el DataFrame transformado y ordenado
display(df_transformed)

Año,item,2029,2030,2031,2033,2040,2050
0,Capacidad instalada Generación,44.821459,44.685659,44.541339,44.541339,48.316181,63.034144
1,Capacidad instalada LDES,0.275808,0.275808,0.275808,0.275808,1.933420,4.406606
2,Capacidad instalada BESS,2.500600,2.500600,2.500600,2.500600,2.520943,3.469428
3,Participación renovable,66.344160,66.545781,66.761398,66.761398,72.266364,73.126217
4,Participación renovable variable,47.250270,47.393864,47.547427,47.547427,54.553540,59.475933
5,Generación bruta,96399.331505,96403.257503,98243.313531,100806.183257,119309.115502,151157.733220
6,Energía movida por LDES,5687.246342,5699.973791,5538.036471,5799.846013,37567.373172,88270.859008
7,Energía movida por BESS,3329.654689,3203.789691,3362.674762,3139.904851,3091.026345,4962.285316
8,Generación renovable energía,72503.119602,72509.264519,73490.979416,74633.713578,93010.304668,116651.344777
9,Generación renovable variable energía,48680.948810,47786.244405,48554.908441,49755.500848,67502.646034,91920.746337
